In [2]:
import pypyodbc as odbc #!pip install pypyodbc
import pandas as pd
import time
import os

print(odbc.drivers())

['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client RDA 11.0', 'SQL Server Native Client RDA 11.0']


#### Needed Inputs:

In [3]:
driverName = "SQL Server" 
serverName = "DESKTOP-UVMKETP\SQLEXPRESS"
dataBaseName = "SQL_PowerBI_Test"
username = "jvenncpe"
pwd = "#Juv1994*"

tableName = "olist_order_items_dataset"
csv_FilePaTH = r"G:\00 Work Files\02 Work Trainings & Apps\00 Project Portfolio\Ecommerce_Amazon (Python, SQL, PowerBI)\CSV Files\Amazon_Olist"
csv_FileName = r"olist_order_items_dataset.csv" #To be used for column name reference in SQL Table
csv_FolderName = r"olist_order_items_dataset"

### Custom Functions

In [4]:
def updateSQL():
    try:
        conn.commit()
        return print("SQL Updated Successfully!")
        
    except Exception as e:
        return print(f"Error occurred: {e}")

def SQL_createTable_columnName(column):
    column_names = []
    for x in column:
        x = f'[{x}] VARCHAR(MAX)'
        column_names.append(x)
    return ', '.join(column_names)  # Join the column names with commas

def SQL_createTable_query(tableName,tableColumns):
    createTable_query = f"\
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{tableName}')\
    BEGIN\
        CREATE TABLE {dataBaseName}.dbo.{tableName}({tableColumns})\
    END".strip()
    return createTable_query

def bulk_insert(datapath_file, SQL_Table):
    insertBulk_query = f"""
        BULK INSERT {SQL_Table}
        FROM '{datapath_file}'
        WITH
        (
            FIRSTROW = 2,
            FIELDTERMINATOR = ',', 
            ROWTERMINATOR = '\n',  
            TABLOCK
        ) 
    """.strip()
    return insertBulk_query

#### Adhoc SQL Configuration

In [5]:
connection_string = f"""
    DRIVER={{{driverName}}};
    SERVER={serverName};
    DATABASE={dataBaseName};
    Trust_Connection=yes;
    UID={username};
    PWD={pwd};
"""
try:
    conn = odbc.connect(connection_string)
    cursor = conn.cursor()
    print("Connection successful!")

except Exception as e:
    print(f"Error: {e}")


Connection successful!


#### Main Function

In [6]:
if __name__ == '__main__':
    csv_filePaTH_REF = os.path.join(csv_FilePaTH,csv_FolderName,csv_FileName)   
    csv_FoldderPath = os.path.join(csv_FilePaTH, csv_FolderName)
    
    df_productDetails = pd.read_csv(csv_filePaTH_REF, encoding='latin1', na_values=[""])
    
    for columnName in df_productDetails.columns:
        columnName_After = columnName.replace(".","")
        columnName_After = columnName_After.replace(" ","_")
        df_productDetails.rename(columns={columnName: columnName_After}, inplace=True)
    
    cursor = conn.cursor()
    tableColumns = SQL_createTable_columnName(df_productDetails.columns)
    
    try:
        with cursor:
            cursor.execute(SQL_createTable_query(tableName,tableColumns))
            updateSQL()
    except Exception as e:
        print(e)
        conn.rollback()
        print('Transaction rollback')
    
    data_files = os.listdir(csv_FoldderPath)
    SQL_Table = f'{dataBaseName}.dbo.{tableName}'
    
    cursor = conn.cursor()
    try:
        with cursor:
            print(f"Target SQL Table: {SQL_Table}")
            print(f"---------------------------")
            print(f"Files to Upload:")
            
            for data_file in data_files:
                if data_file.endswith('.csv'):
                    cursor.execute(bulk_insert(os.path.join(csv_FoldderPath, data_file), SQL_Table))
                    print(data_file)
                    
            print(f"---------------------------")
            updateSQL()
            
    except Exception as e:
        print(e)
        conn.rollback()
        print('Transaction rollback')

SQL Updated Successfully!
Target SQL Table: SQL_PowerBI_Test.dbo.olist_order_items_dataset
---------------------------
Files to Upload:
olist_order_items_dataset.csv
---------------------------
SQL Updated Successfully!


In [7]:
df_productDetails.head()

,ï»¿order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,19/09/2017 9:45,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,03/05/2017 11:05,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,18/01/2018 14:48,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,15/08/2018 10:10,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,13/02/2017 13:57,199.90,18.14
